In [1]:
import numpy as np
import time
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(sparse= False)

In [97]:
%ls data

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: D4E0-269A

 C:\Users\Nayo\dl\data 디렉터리

2021-08-12  오후 08:53    <DIR>          .
2021-08-12  오후 08:53    <DIR>          ..
2019-09-20  오후 06:24           191,873 abalone.data.csv
               1개 파일             191,873 바이트
               2개 디렉터리  219,660,664,832 바이트 남음


In [63]:
np.__version__

'1.18.1'

In [65]:
np.random.seed(1234)
np.random.seed((time.time()))

TypeError: Cannot cast array from dtype('float64') to dtype('int64') according to the rule 'safe'

In [9]:
rnd_mean = 0
rnd_std = 0.003
learning_rate = 0.001

In [10]:
def abalone_exec(epoch_count = 10, mb_size = 10, report = 1):
    load_abalone_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)
    

def load_abalone_dataset():
    global data, input_cnt, output_cnt
    data = pd.read_csv("./data/abalone.data.csv", names = ["Sex", "Length", "Diameter","Height","Whole weight",
                                                           "Shucked weight","Viscera weight", "Shell weight", "Rings"], header = 0)
    ohe = OneHotEncoder(sparse = False)
    ohe.fit(data[["Sex"]])
    ohe_columns = []
    for col in ohe.categories_:
        ohe_columns += col.tolist()

    ohe_data = ohe.transform(data[["Sex"]])
    data = pd.concat([pd.DataFrame(ohe_data, columns = ohe_columns), data,], axis = 1)
    data = data.drop("Sex", axis = 1)
    input_cnt, output_cnt = 10,1
    data = np.array(data)
    
    
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(rnd_mean, rnd_std, [input_cnt, output_cnt])
    bias = np.zeros([output_cnt])
    
    
def train_and_test(epoch_count, mb_size, report):
    step_count = arrange_data(mb_size)
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], []
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss)
            accs.append(acc)
            
        if report > 0 and (epoch+1) % report == 0:
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))
    

def arrange_data(mb_size):
    global data, shuffle_map, test_begin_idx
    shuffle_map = np.arange(data.shape[0])
    np.random.shuffle(shuffle_map)
    step_count = int(data.shape[0] * 0.8) // mb_size
    test_begin_idx = step_count * mb_size
    return step_count

def get_test_data():
    global data, shuffle_map, test_begin_idx, output_cnt
    test_data = data[shuffle_map[test_begin_idx:]]
    return test_data[:, :-output_cnt], test_data[:, -output_cnt:]

def get_train_data(mb_size, nth):
    global data, shuffle_map, test_begin_idx, output_cnt
    if nth == 0:
        np.random.shuffle(shuffle_map[:test_begin_idx])
    train_data = data[shuffle_map[mb_size*nth:mb_size*(nth+1)]]
    return train_data[:, :-output_cnt], train_data[:, -output_cnt:]


def run_train(x, y):
    output, aux_nn = forward_neuralnet(x)
    loss, aux_pp = forward_postproc(output, y)
    accuracy = eval_accuracy(output, y)
    
    G_loss = 1.0
    G_output = backprop_postproc(G_loss, aux_pp)
    backprop_neuralnet(G_output, aux_nn)
    
    return loss, accuracy

def run_test(x, y):
    output, _ = forward_neuralnet(x)
    accuracy = eval_accuracy(output, y)
    return accuracy


def forward_neuralnet(x):
    global weight, bias
    output = np.matmul(x, weight) + bias
    return output, x

def backprop_neuralnet(G_output, x):
    global weight, bias
    g_output_w = x.transpose()
    
    G_w = np.matmul(g_output_w, G_output)
    G_b = np.sum(G_output, axis=0)

    weight -= learning_rate * G_w
    bias -= learning_rate * G_b
    
    
def forward_postproc(output, y):
    diff = output - y
    square = np.square(diff)
    loss = np.mean(square)
    return loss, diff

def backprop_postproc(G_loss, diff):
    shape = diff.shape
    
    g_loss_square = np.ones(shape) / np.prod(shape)
    g_square_diff = 2 * diff
    g_diff_output = 1

    G_square = g_loss_square * G_loss
    G_diff = g_square_diff * G_square
    G_output = g_diff_output * G_diff
    
    return G_output


def eval_accuracy(output, y):
    mdiff = np.mean(np.abs((output - y)/y))
    return 1 - mdiff


def backprop_postproc_oneline(G_loss, diff):  
    return 2 * diff / np.prod(diff.shape)

In [36]:
abalone_exec()

Epoch 1: loss=33.625, accuracy=0.557/0.806
Epoch 2: loss=8.267, accuracy=0.819/0.815
Epoch 3: loss=7.621, accuracy=0.813/0.811
Epoch 4: loss=7.506, accuracy=0.811/0.810
Epoch 5: loss=7.425, accuracy=0.810/0.810
Epoch 6: loss=7.357, accuracy=0.811/0.810
Epoch 7: loss=7.299, accuracy=0.810/0.810
Epoch 8: loss=7.249, accuracy=0.810/0.810
Epoch 9: loss=7.204, accuracy=0.810/0.810
Epoch 10: loss=7.165, accuracy=0.810/0.810

Final Test: final accuracy = 0.810


In [37]:
print(weight)
print(bias)

[[1.64493007]
 [1.01652812]
 [1.48968972]
 [2.0251828 ]
 [1.6070802 ]
 [0.58507356]
 [2.38338352]
 [0.53524534]
 [0.46649974]
 [1.04185969]]
[4.15683041]


In [7]:

LEARNING_RATE = 0.1
abalone_exec(epoch_count=100, mb_size=100, report=20)

Epoch 20: loss=7.721, accuracy=0.816/0.820
Epoch 40: loss=7.462, accuracy=0.808/0.816
Epoch 60: loss=7.327, accuracy=0.808/0.817
Epoch 80: loss=7.226, accuracy=0.808/0.817
Epoch 100: loss=7.145, accuracy=0.808/0.818

Final Test: final accuracy = 0.818
